In [1]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from tqdm import tqdm
from pprint import pprint

In [ ]:
import os 

os.environ["HUGGINGFACEHUB_API_TOKEN"]=''

In [3]:
class LLMState(TypedDict):
    question : str
    answer : str

In [4]:
llm = HuggingFaceEndpoint(
    repo_id="moonshotai/Kimi-K2-Instruct-0905",
    task="text-generation",
    temperature=0.6,  # Recommended by Moonshot AI
    max_new_tokens=4096,
)

chat_model = ChatHuggingFace(
    llm = llm
)


def llm_node(state: LLMState) -> LLMState:
    que = state['question']

    messages = [
        {"role": "system", "content": "You are a great teacher just like Richard Feymann, you have to teach python concepts with indepth knowledge and with multiple examples ranging from basic ones to advanced ones"},
        {"role": "user", "content": que}
    ]

    ans = chat_model.invoke(messages).content

    state['answer'] = ans

    return state

/home/rajgajjar04/Learnings/Langchain_Langgraph/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
graph = StateGraph(LLMState)

graph.add_node("llm_node", llm_node)

graph.add_edge(START, "llm_node")
graph.add_edge("llm_node", END)

workflow = graph.compile()

In [6]:
initial_state = {"question": "Teach me about python decorators in depth"}

final_state = workflow.invoke(initial_state)

print(final_state['answer'])

Below is a mini-textbook on Python decorators.  
Read it in order: each section adds a new layer.  
If you only remember one sentence, make it this:

> “A decorator is just a callable that takes a callable and returns a callable; everything else is optional convenience.”

--------------------------------------------------------------------
1.  The Naked Truth – no syntax sugar yet
--------------------------------------------------------------------
A decorator is **any callable** that accepts a function and returns a function.

```python
def loud(fn):              # 1. a decorator
    def wrapper(*a, **kw):
        print('About to call', fn.__name__)
        result = fn(*a, **kw)
        print('Finished')
        return result
    return wrapper          # 2. returns a new function

def greet(name):
    print('Hello', name)

greet = loud(greet)          # 3. manual decoration
greet('Bob')
```

Output
```
About to call greet
Hello Bob
Finished
```

`@` is only **syntactic sugar** for th